imports:

In [ ]:
ympy_path = 'C:/Users/elgui/Documents/Emr Lab Post Doc/Bioinformatics and Modelling/Python/EMRyeast/YeastMorphologyPypeline/'

import sys
sys.path
sys.path.append(ympy_path)

import pickle
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2

import ympy

Enter terms that depend on precise experiment design

In [ ]:
user_enter_folder_path = #path to your raw image data
user_enter_experiment_name = #a short name
user_enter_measured_protein_name = #for example, 'Art1-mNG'
user_enter_marker_name =#for example, 'golgi'
#use a 6 character id prefix on all images (or change exp_ID_loc)
user_enter_expid_lookup = {
        'column_names': ['strain', 'condition'],
        '######':('%%%','$$$') # where ######, %%% and $$$ are replaced with expid, strain and 
                               # condition, respectively, for each such tuple
        }

Populate full experiment parameters dictionary; using terms defined in above block. Additinal defaults are supplied below; change as needed

In [ ]:
exp_params = dict(
        folder_path = user_enter_folder_path,
        experiment_name = user_enter_experiment_name,
        image_extension = 'R3D_D3D.dv', 
        image_reader = ympy.helpers.basicDVreader,
        reader_args = {'n_channels': 3, 'z_first': True}, 
        measured_protein_name = user_enter_measured_protein_name, 
        marker_name = user_enter_marker_name, 
        measured_protein_channel = 1, 
        measured_protein_z = 3, 
        marker_channel = 0, 
        bf_offest_vector = [2, -1],
        exp_ID_loc = [0,6], # start position, length
        expid_lookup = user_enter_expid_lookup
        )

In [ ]:
pipeline = ympy.pipelines.GFPwMarkerPipeline(exp_params)
pipeline.runPipeline()

In [ ]:
pipeline_results_path = pipeline.save_path
print(pipeline)
qcpipeline = ympy.pipelines.QualityControlPipeline(pipeline.save_path, exp_params)
qcpipeline.firstTimeStartInPipeline()
qcpipeline.prefilterQC()
qcpipeline.callQCframeLib()
qcpipeline.QCframeLib.initializeLibrary()

In [ ]:
qcpipeline.autosave = False
qcpipeline.QCframeLib.displayForQC()

In [ ]:
qcpipeline.expandExpIDs()
qcpipeline.sortQCDF()

In [ ]:
norm_cortex_series = pd.Series(qcpipeline.accepted_df.loc[:,'cortex(non-{})_{}_histogram-dif'
                                                          .format(user_enter_marker_name, user_enter_measured_protein_name)]
                               /qcpipeline.accepted_df.loc[:,'total_cell_{}_histogram-sum'
                                                           .format(user_enter_measured_protein_name)])
norm_marker_series = pd.Series(qcpipeline.accepted_df.loc[:,'{}(ciricles)_{}_histogram-dif'
                                                          .format(user_enter_marker_name, user_enter_measured_protein_name)] /
                       qcpipeline.accepted_df.loc[:,'total_cell_{}_histogram-sum'
                                                  .format(user_enter_measured_protein_name)])

df = qcpipeline.accepted_df.assign(norm_cortex = norm_cortex_series)
df = df.assign(norm_marker = norm_marker_series)